---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

## **Out Parser**

### 1) 개념

* LLM의 출력을 더 유용하고, 구조화된 형태로 변환하는 중요한 컴포넌트

* 역할
  * `LLM`의 출력 → 더 적합한 형식으로 변환
  * **구조화된 데이터 생성** 에 매우 유용
  * `LangChain 프레임워크`에서 다양한 종류의 출력 데이터를 파싱하고 처리

* 주요 특징
  * **다양성**: LangChain은 많은 종류의 출력 파서를 제공
  * **스트리밍 지원**: 많은 출력 파서들이 스트리밍을 지원
  * **확장성**: 최소한의 모듈부터 복잡한 모듈까지 확장 가능한 인터페이스를 제공
  
* 이점
  * **구조화**: `LLM`의 `자유 형식 텍스트` 출력 → `구조화된 데이터` 
  * **일관성**: 출력 형식을 일관되게 유지 → 후속 처리 용이하게 함
  * **유연성**: 다양한 출력 형식(`JSON`, `list`, `dict`등)으로 변환 가능
  
* 사용 전 / 후 차이 
  * 아무런 출력 파서를 사용하지 않을 때
  
  ```markdown
    **중요 내용 추출:**

    1. **발신자:** 김철수 (chulsoo.kim@bikecorporation.me)
    2. **수신자:** 이은채 (eunchae@teddyinternational.me)
    3. **제목:** "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
    4. **요청 사항:**
      - ZENESIS 모델의 상세한 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함)
    5. **미팅 제안:**
      - 날짜: 다음 주 화요일 (1월 15일)
      - 시간: 오전 10시
      - 장소: 귀사 사무실

    6. **발신자 정보:**
      - 김철수, 상무이사, 바이크코퍼레이션
  ```

---

  * **`JSON`** 형식의 구조화된 답변

  ```json
    {
    "person": "김철수",
    "email": "chulsoo.kim@bikecorporation.me",
    "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
    "summary": "바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 신규 자전거 'ZENESIS' 모델에 대한 브로슈어 요청과 기술 사양, 배터리 성능, 디자인 정보 요청. 또한, 협력 논의를 위해 1월 15일 오전 10시에 미팅 제안.",
    "date": "1월 15일 오전 10시"
    }
  ```

---

### **`2) Pydantic 출력 파서 (PydanticOutputParser)`**

* `PydanticOutputParser` 는 **`언어 모델의 출력을 더 구조화된 정보로 변환`** 하는 데 도움이 되는 클래스
  * 단순 텍스트 형태의 응답 대신, 사용자가 필요로 하는 정보를 명확하고 체계적인 형태로 제공 가능
  * 언어 모델의 출력을 특정 데이터 모델에 맞게 변환 → 더 용이하게 정보를 처리하고 활용 가능

<br>

* **두 가지 핵심 메서드**
  * `get_format_instructions()`: 
    * 언어 모델이 출력해야 할 **`정보의 형식`** 을 정의하는 지침(instruction) 을 제공
    * 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할
    * 이때 설정하는 지침(`instruction`) 의 역할이 매우 중요 → 언어 모델은 출력을 구조화하고, 이를 특정 데이터 모델에 맞게 변환
  * `parse()`: 
    * 언어 모델의 출력(문자열로 가정)을 받아들여 이를 **`특정 구조`** 로 분석하고 변환합니다. 
    * **`Pydantic`** 와 같은 도구를 사용하여, `입력된 문자열`을 사전 정의된 `스키마`에 따라 `검증`하고, 해당 스키마를 따르는 `데이터 구조`로 `변환`

<br>

* *[Pydatic 공식 도큐먼트](https://docs.pydantic.dev/latest/)*

In [1]:
# 1_새 프롬프트 생성하기
from langsmith import Client
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langsmith import Client

import os
import json


# 클라이언트 생성 
api_key = os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=api_key)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langsmith import traceable

# LLM 초기화
gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0.7,                                    
        max_output_tokens=4096,
    )

* [이메일 본문 예시](https://wikidocs.net/233786)

In [3]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [ ]:
# 출력 파서를 사용하지 않는 경우
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from itertools import chain

gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0.7,                                    
        max_output_tokens=4096,
    )

# 중요한 내용을 추출하기 위한 프롬프트 템플릿을 정의합니다.
prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

# LangChain의 Expression Language (LCEL)를 사용하여 체인을 구성합니다.
# 프롬프트 -> 모델 -> 문자열 출력 파서 순서로 연결됩니다.
# StrOutputParser를 추가하면 모델의 응답을 깔끔한 문자열로 변환합니다.
chain = prompt | gemini_lc | StrOutputParser()

# stream() 메서드를 사용하여 응답을 실시간으로 스트리밍합니다.
print("--- 이메일 내용 요약 스트리밍 ---")
print("------------------------------\n")
for chunk in chain.stream({"email_conversation": email_conversation}):
    print(chunk, end="", flush=True)

print("\n\n--- 전체 요약 결과 ---")
print("--------------------\n")
# invoke() 메서드를 사용하여 전체 응답을 한 번에 받습니다.
answer = chain.invoke({"email_conversation": email_conversation})
print(answer)

<small>

* 셀 출력 (temperature=0, outparser X)(2.8s)

    ```markdown
    --- 이메일 내용 요약 스트리밍 ---
    ------------------------------

    이메일의 중요 내용은 다음과 같습니다.

    *   **발신자:** 김철수 상무 (바이크코퍼레이션)
    *   **수신자:** 이은채 대리 (테디인터내셔널)
    *   **주요 목적:** 테디인터내셔널의 신규 자전거 "ZENESIS" 유통 협력 제안
    *   **요청 사항:**
        *   "ZENESIS" 자전거의 상세 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함)
    *   **제안 사항:**
        *   협력 논의를 위한 미팅 제안
        *   **미팅 일정:** 다음 주 화요일 (1월 15일) 오전 10시
        *   **미팅 장소:** 테디인터내셔널 사무실

    --- 전체 요약 결과 ---
    --------------------

    이메일에서 추출된 중요한 내용은 다음과 같습니다.

    *   **발신자:** 김철수 상무 (바이크코퍼레이션)
    *   **수신자:** 이은채 대리 (테디인터내셔널)
    *   **주요 목적:** 테디인터내셔널의 신규 자전거 "ZENESIS"에 대한 유통 협력 제안
    *   **요청 사항:**
        *   "ZENESIS" 자전거의 상세 브로슈어 요청 (기술 사양, 배터리 성능, 디자인 정보 포함)
    *   **제안 사항:**
        *   협력 논의를 위한 미팅 제안: **다음 주 화요일 (1월 15일) 오전 10시**, 테디인터내셔널 사무실
    ```

In [ ]:
# outparser 사용
from itertools import chain
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser,  PydanticOutputParser

class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

# instruction 출력해보기
print(parser.get_format_instructions())

<small>

* 셀 출력 (0.0s)

    ```markdown

    The output should be formatted as a JSON instance that conforms to the JSON schema below.

    As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
    the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

    Here is the output schema:
    ```

    ```json
    {"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
    ```


* **프롬프트** 정의

  * `question`: 유저의 질문 받기
  * `email_conversation`: 이메일 본문의 내용 입력
  * `format`: 형식 지정

In [10]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())


* `Chain` 생성하기

In [20]:
from google import genai
from langchain_google_genai import ChatGoogleGenerativeAI

client = genai.Client()

# chain 생성
chain = prompt | gemini_lc

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


* `Chain` 실행 → **결과 확인**

In [ ]:
import json
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from pydantic import BaseModel, Field

# gemini_lc 변수에 적절한 모델 객체를 할당해야 함
# genai.GenerativeModel을 ChatGoogleGenerativeAI로 래핑하여 사용하기
# API 키가 설정되어 있어 햠
# genai.configure(api_key="YOUR_API_KEY")
gemini_lc = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0.7,                                    
        max_output_tokens=4096,
    )


# 이메일 내용
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

# Pydantic 모델 정의
# 이메일에서 추출하고 싶은 정보의 구조를 클래스로 정의하기
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

# PydanticOutputParser 생성
# 모델의 응답을 위에서 정의한 Pydantic 객체로 파싱하는 파서를 만듭니다.
parser = PydanticOutputParser(pydantic_object=EmailSummary)

# instruction 출력하기
print(parser.get_format_instructions())

<small>

* 셀 출력

    ```markdown
    The output should be formatted as a JSON instance that conforms to the JSON schema below.

    As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
    the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

    Here is the output schema:
    ```
    ```json
    {"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
    ```

In [48]:
# 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)
# 프롬프트에 JSON 형식 지시사항 추가
# models.generate_content는 JSON을 출력하도록 명시적으로 지시하기
# parser.get_format_instructions()가 그 역할을 함
# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())


In [49]:
# chain 생성하기
chain = prompt | gemini_lc

# Chain 실행 → 결과 출력
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용 중 주요 내용을 추출해 주세요.",
    }
)

# 결과 = JSON 형태로 출력하기
output = stream_response(response, return_output=True)

In [50]:
print(type(output))

<class 'str'>


* **`parser`** 추가 체인 생성하기

In [53]:
chain = prompt | gemini_lc | parser

In [54]:
# chain 실행 → 결과 출력
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과 = EmailSummary = 객체 형태 출력
response

EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 신규 자전거 "ZENESIS"에 대한 유통 협력을 제안하며, 상세 브로슈어(기술 사양, 배터리 성능, 디자인)를 요청했습니다.', date='다음 주 화요일(1월 15일) 오전 10시')

<small>

* 셀 출력 (1.4s)

    ```markdown
    EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='바이크코퍼레이션의 김철수 상무가 테디인터내셔널의 이은채 대리에게 신규 자전거 "ZENESIS"에 대한 유통 협력을 제안하며, 상세 브로슈어(기술 사양, 배터리 성능, 디자인)를 요청했습니다.', date='다음 주 화요일(1월 15일) 오전 10시')
    ```


### 3) **`with_structurec_output()`**

* `.with_structured_output(Pydantic)` → 출력 파서를 추가 → 출력을 `Pydantic 객체`로 변환
* *한 가지 아쉬운 점은 `strea()` 기능을 지원하지 않는다는 점*

In [55]:
gemini_lc_with_structered = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0.7,                                    
        max_output_tokens=4096,
    ).with_structured_output(EmailSummary)

In [ ]:
# invoke() 함수를 호출 → 결과 출력
answer = gemini_lc_with_structered.invoke(email_conversation)
answer

<small>

* 셀 출력 (변수 조절 = 위와 같음)(1.3s)

    ```markdown
    EmailSummary(person='김철수', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary='"ZENESIS" 자전거의 유통 협력과 관련하여 상세 브로슈어를 요청하며, 1월 15일 오전 10시에 미팅을 제안하는 내용.', date='1월 15일 오전 10시')
    ```